In [2]:
sid = '1510700'
name = 'Nguyen Thanh Dat'

import tensorflow as tf
import numpy as np
import sklearn.model_selection as sk


# Bài 1
Trong ví dụ ở trên, chúng ta đã sử dụng toàn bộ tập dữ liệu để train, và đồng thời cũng lấy chính dữ liệu đó để test, điều đó
dẫn tới trường hợp "học khớp" (overfitting).
Sinh viên hãy chia tập dữ liệu ra làm 2 phần riêng biệt ~90% để train và ~10% còn lại dùng để test. (sửa trực tiếp ở code ví
dụ ở trên)
Hãy cho biết kích thước của mỗi tập sau khi tách ra
Hãy cho biết các nhận xét của kết quả khi chia ra 2 tập rời như vậy

In [5]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# load dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1
# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
'b1': tf.Variable(tf.random_normal([n_hidden_1])),
'b2': tf.Variable(tf.random_normal([n_hidden_2])),
'out': tf.Variable(tf.random_normal([n_classes]))
}
# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
            # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")
    # Test model
    pred = tf.nn.softmax(logits) # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost=312.579218864
Epoch: 0002 cost=95.374959994
Epoch: 0003 cost=70.981094340
Epoch: 0004 cost=56.736979911
Epoch: 0005 cost=47.729590488
Epoch: 0006 cost=42.816328885
Epoch: 0007 cost=37.214596614
Epoch: 0008 cost=34.447859274
Epoch: 0009 cost=31.783540420
Epoch: 0010 cost=29.109287397
Epoch: 0011 cost=27.610781347
Epoch: 0012 cost=26.021569273
Epoch: 0013 cost=24.704048311
Epoch: 0014 cost=22.992715377
Epoch: 0015 cost=21.620997228
Epoch: 0016 cost=21.313882766
Epoch: 0017 cost=20.156914326
Epoch: 0018 cost=19.255096169
Epoch: 0019 cost=19.066982886
Epoch: 0020 cost=18.191530964
Epoch: 0021 cost=17.139487472
Epoch: 0022 cost=16.885087367
Epoch: 0023 cost=16.404754116
Epoch: 0024 cost=15.783774483
Epoch: 0025 cost=15.497318083
Epoch: 0026 cost=15.584400601
Epoch: 0027 cost=15.234909901

In [28]:
print("size of train",len(mnist.train.images))

print("size of test",len(mnist.test.images))


size of train 55000
size of test 10000


### Answer:
Trong tập dữ liệu trên đã chia ra 2 tập train và test có tỉ lệ 5,5:1. Sau khi test thì kết qủa là Accuracy ~ 0.876 tương đối tốt

# Bài 2
Hãy sử dụng tập dữ liệu cifar 10 (https://www.cs.toronto.edu/~kriz/cifar.html)
Hãy xử lý dữ liệu, thay đổi model (nếu cần) và điều chỉnh các thông số cần thiết phù hợp với dữ liệu này.
Sinh viên hãy train, test và cho biết kết quả đạt được (yêu cầu giữ kết quả output khi chạy train, test lúc nộp bài ở các cell tiếp theo ở dưới

In [25]:
# Import  data
import cifar10



ModuleNotFoundError: No module named 'cifar10'